# Project Title
### Data Engineering Capstone Project

#### Project Summary
In this project we will collect data sets and create tables which will allow to assess the influence of solar flares on weathern patterns on Earth. An investigation showed that small fluctuations in spot coverage can influence the climate.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import os, glob
import shutil
import tempfile
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

### Step 1: Scope the Project and Gather Data

#### Scope 
In order to evaluate the influence of sunspot coverage on earth weather patter we combined data from two sources. For one, we need the time series for the sun spot coverage and the other data which logs the weather.


#### Describe and Gather Data 
Two data sets are used:

* Sunspots ([link](https://www.kaggle.com/robervalt/sunspots)): This dataset is a csv file with the date and the montly mean total sunspot number as obtained by the Database from SIDC - Solar Influences Data Analysis Center - the solar physics research department of the Royal Observatory of Belgium.

* NOAA GSOD ([link](https://data.noaa.gov/dataset/dataset/global-surface-summary-of-the-day-gsod)): This is a public data set from the National Oceanic and Atmospheric Administration (NOAA). This data set is very large.

### Data exploration:

We start by understanding the content of the data.

In [2]:
#create spark session
spark = SparkSession.builder\
        .appName("Comparing solar flares with global weathern pattern")\
        .enableHiveSupport().getOrCreate()



Read sunspot csv file.

In [3]:
#csvFile = 'hessi.solar.flare.2002to2016.csv'
csvFile = 'Sunspots.csv'
dfSpots = spark.read.csv(csvFile, header=True, inferSchema=True)


Print the schema, show the size of the data and the date range.

In [4]:
dfSpots.printSchema()
print(dfSpots.count())
print(dfSpots.head())
dfSpots.select([min("Date")]).show()
dfSpots.select([max("Date")]).show()

root
 |-- _c0: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Monthly Mean Total Sunspot Number: double (nullable = true)

3252
Row(_c0=0, Date=datetime.datetime(1749, 1, 31, 0, 0), Monthly Mean Total Sunspot Number=96.7)
+-------------------+
|          min(Date)|
+-------------------+
|1749-01-31 00:00:00|
+-------------------+

+-------------------+
|          max(Date)|
+-------------------+
|2019-12-31 00:00:00|
+-------------------+



NOAA data is available as `tar.gz` files for each year. We start by exploring only one year. Each `tar.gz` file has several csv files. As an example we start by showing just one file.

In [5]:
#create a temporary directory where the data will be extracted and explored
with tempfile.TemporaryDirectory() as temp_directory:
    #unpack to year 2016 data to temporary directory
    shutil.unpack_archive('/home/workspace/NOAA_data/2016.tar.gz', temp_directory)
    #get a list of all csv files
    csvFiles = glob.glob( os.path.join(temp_directory,'*.csv') )
    #read the first csv file and print some information
    dfNOAA = spark.read.csv(csvFiles[0], header=True, inferSchema=True)
    dfNOAA.printSchema()
    print(dfNOAA.count())
    print(dfNOAA.head())

root
 |-- STATION: long (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- TEMP_ATTRIBUTES: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- DEWP_ATTRIBUTES: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- SLP_ATTRIBUTES: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- STP_ATTRIBUTES: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- VISIB_ATTRIBUTES: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- WDSP_ATTRIBUTES: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MAX_ATTRIBUTES: string (nullable = true)
 |-- MIN: double (nullable = true)
 |-- MIN_ATTRIBUTES: string (nullable = true)
 |-- PRCP: double (nullable = 

### Step 2: Explore and Assess the Data
#### Explore the Data 

***Columns of interest for NOAA data:***

* STATION: Station number (WMO/DATSAV3 number) for the location

* DATE: Date of reading

* LATITUDE: Station latitude

* LONGITUDE: Station longitude

* ELEVATION: Station elevation

* NAME: Station name

* TEMP: Temperature reading

* DEWP: Dew Point reading

* SLP: Mean sea level pressure for the day in millibars to tenths. Missing = 9999.9

* STP: Mean station pressure for the day in millibars to tenths. Missing = 9999.9

* VISIB: Mean visibility for the day in miles to tenths. Missing = 999.9

* WDSP: Mean wind speed for the day in knots to tenths. Missing = 999.9

* MXSPD: Maximum sustained wind speed reported for the day in knots to tenths. Missing = 999.9

* GUST: Maximum wind gust reported for the day in knots to tenths. Missing = 999.9

* MAX: Maximum temperature reported during the day in Fahrenheit to tenths--time of max temp report varies by country and region.

* MIN: Minimum temperature reported during the day in Fahrenheit to tenths--time of min temp report varies by country and region.

* PRCP: Total precipitation (rain and/or melted snow) reported during the day in inches and hundredths.


***Columns of interest for Sun Spot data:***

* _c0: Idenfier
* Date: Date
* Monthly Mean Total Sunspot Number

#### Cleaning Steps

For the Sunspot data we only drop duplicate entries and separete date and time column.

For the NOAA data we select only columns of interestes, we drop duplicates **and** filter missing values. We iterate over all `tar.gz` files and read all csv files in to a spark dataframe. We will save our staged result in a parquet file.

In [6]:
# READ sunspot csv file, drop duplicates and add columns dayofmonth, month and year.
csvFile = 'Sunspots.csv'
dfSpots = spark.read.csv(csvFile, header=True, inferSchema=True)

spotsTable = dfSpots.select(col('_c0').alias('spotId'), 
                            date_format(dfSpots["Date"], 'yyyy-MM-dd').alias('Date'),
                            date_format(dfSpots["Date"], 'h:m:s a').alias('Time'),
                            col('Monthly Mean Total Sunspot Number').alias('SunspotNumber'),)\
                            .dropDuplicates()
# Show schema for verfication propuses.
spotsTable.printSchema()
# Show number of counts. This table actually has no duplicates.
spotsTable.count()

root
 |-- spotId: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- SunspotNumber: double (nullable = true)



3252

In [7]:
spotsTable.head()

Row(spotId=284, Date='1772-09-30', Time='12:0:0 AM', SunspotNumber=84.2)

In [8]:
# READ NOAA data.
noaaFiles = glob.glob('NOAA_data/*.tar.gz') # List of all tar.gz files
for tarfile in noaaFiles[0:3]:
    with tempfile.TemporaryDirectory() as temp_directory:
        #unpack
        shutil.unpack_archive(tarfile, temp_directory)
        csvFiles = glob.glob( os.path.join(temp_directory,'*.csv') )
        #read all csv files into spark dataframe
        dfNOAA = spark.read.csv(csvFiles, header=True, inferSchema=True)
        #clean up the data
        noaaTable = dfNOAA.select('STATION', date_format(dfNOAA["DATE"], 'yyyy-MM-dd').alias('Date'),
                                date_format(dfNOAA["DATE"], 'h:m:s a').alias('Time'),
                                'LATITUDE', 'LONGITUDE', 'ELEVATION',
                                'NAME', 'TEMP', 'DEWP', 'SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 
                                'GUST', 'MAX', 'MIN', 'PRCP').dropDuplicates()\
                                .filter("SLP != 9999.9")\
                                .filter("STP != 9999.9")\
                                .filter("VISIB != 999.9")\
                                .filter("WDSP != 999.9")\
                                .filter("MXSPD != 999.9")\
                                .filter("GUST != 999.9")
        #Append the data to a parquet file
        noaaTable.write.mode('append').parquet("NOAA_data.parquet")



Check that parquet file has been created and the schema is correct.

In [12]:
noaaParquet = spark.read \
                .format('parquet') \
                .load('NOAA_data.parquet')

In [13]:
noaaParquet.printSchema()
print(noaaParquet.head())
noaaParquet.count()

root
 |-- STATION: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MIN: double (nullable = true)
 |-- PRCP: double (nullable = true)

Row(STATION=72532604894, Date='2016-04-24', Time='12:0:0 AM', LATITUDE=41.74278, LONGITUDE=-89.67611, ELEVATION=197.2, NAME='STERLING ROCK FALLS WHITESIDE CO JOSH BITTORF FIELD AIRPORT, IL US', TEMP=60.9, DEWP=48.4, SLP=1030.0, STP=999.9, VISIB=9.5, WDSP=5.3, MXSPD=9.9, GUST=15.0, MAX=80.1, MIN=44.6, PRCP=0.0)


1097077

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
We use a NoSQL data model, because of the very large NOAA data set. 

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

After cleaning the data, we will create a final table which can be queried for each months and it will return sun spot coverage and average weather for each site and globally.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model. We start by aggregating the NOAA parquet data and averaging over the weather data.

In [27]:
aggNOAAdata = noaaParquet.groupby('Date').agg( 
                                col('Date').cast('timestamp').alias('ts'),
                                avg('TEMP').alias('avgTEMP'),
                                avg('DEWP').alias('avgDEWP'), 
                                avg('SLP').alias('avgSLP'), 
                                avg('STP').alias('avgSTP'), 
                                avg('VISIB').alias('avgVISIB'), 
                                avg('WDSP').alias('avgWDSP'), 
                                avg('MXSPD').alias('avgMXSPD'), 
                                avg('GUST').alias('avgGUST'), 
                                avg('MAX').alias('avgMAX'), 
                                avg('MIN').alias('avgMIN'), 
                                avg('PRCP').alias('avgPRCP'), 
                                )

In [28]:
aggNOAAdata.head()

Row(Date='2016-08-17', ts=datetime.datetime(2016, 8, 17, 0, 0), avgTEMP=71.27573149741825, avgDEWP=74.31652323580032, avgSLP=1014.8262478485369, avgSTP=603.6166092943199, avgVISIB=11.279948364888126, avgWDSP=6.893631669535283, avgMXSPD=14.56196213425129, avgGUST=22.15275387263339, avgMAX=82.72185886402751, avgMIN=60.264802065404474, avgPRCP=5.053485370051633)

In [16]:
aggNOAAdata.count()

1096

We now joint the NOAA and sunspot data.

In [31]:
dfSpotWeather = spotsTable.join(aggNOAAdata, 
                (spotsTable.Date == aggNOAAdata.Date) ).drop( aggNOAAdata.Date ).drop(spotsTable.Time)

In [32]:
dfSpotWeather.printSchema()

root
 |-- spotId: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- SunspotNumber: double (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- avgTEMP: double (nullable = true)
 |-- avgDEWP: double (nullable = true)
 |-- avgSLP: double (nullable = true)
 |-- avgSTP: double (nullable = true)
 |-- avgVISIB: double (nullable = true)
 |-- avgWDSP: double (nullable = true)
 |-- avgMXSPD: double (nullable = true)
 |-- avgGUST: double (nullable = true)
 |-- avgMAX: double (nullable = true)
 |-- avgMIN: double (nullable = true)
 |-- avgPRCP: double (nullable = true)



#### 4.2 Data Quality Checks
 
Run Quality Checks

We check that we have no NULL elements in the table:

In [24]:
# Check that key fields have valid values (no nulls or empty)
dfSpotWeather.createOrReplaceTempView("SpotWeatherTable")
SpotWeather_check = spark.sql("""
    SELECT  COUNT(*)
    FROM SpotWeatherTable
    WHERE Date IS NULL OR SunSpotNumber IS NULL 
""")
SpotWeather_check.show(1)
SpotWeather_check.collect()[0][0]

+--------+
|count(1)|
+--------+
|       0|
+--------+



0

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

spotId: Spot identifier
Date: Date of observatoins
SunspotNumber: number of spots
ts: Date as timestamp
avgTEMP: average temperature
avgDEWP: average dew point
avgSLP: average sealevel pressure
avgSTP: average site pressure
avgVISIB: average visibility
avgWDSP: average windspeed
avgMXSPD: average max. windspeed
avgGUST: average wind gust
avgMAX: average max. temp.
avgMIN: average min. temp.
avgPRCP: average precitipitation

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

Given the already large data set Spark is very usuful to deal with it. 

The data should be updated on a yearly basis.

I choose NoSQL as data model which is good for linear scalbility. Increasing the data set posese no problems. The Spark framework allows access for hundreds of people.